In [ ]:
import time
import pandas as pd

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser

In [37]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

---

In [261]:
from selenium import webdriver
import datetime
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(1)
driver.get('https://fair.gg.go.kr/fran/search/searchList.do')
driver.implicitly_wait(1)

In [262]:
driver.find_element_by_xpath('//*[@id="labelList1_1"]').click() #대분류 클릭
time.sleep(1) 
driver.find_element_by_xpath('//*[@id="labelList1_1"]/option[2]').click()#외식 클릭
time.sleep(1) 

driver.find_element_by_xpath('//*[@id="labelList1_2"]').click()#중분류 클릭
time.sleep(1) 
driver.find_element_by_xpath('//*[@id="labelList1_2"]/option[16]').click()#외식 전체 클릭
time.sleep(1) 

driver.find_element_by_xpath('//*[@id="brandCompare"]/div[1]/div/div[3]/div[2]/select').click()#개수
time.sleep(1) 
driver.find_element_by_xpath('//*[@id="brandCompare"]/div[1]/div/div[3]/div[2]/select/option[3]').click()#50개씩
time.sleep(1) 


In [259]:
# 해당 페이지의 표 데이터를 df로 저장하는 함수
def crawling_page():
    global df
    # 페이지를 soup 객체로 전환
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    temp = soup.find_all("tbody")
    # 데이터를 데이터프레임으로 만들어서 df로 저장
    p = parser.make2d(temp[0])
    df = pd.DataFrame(p, columns= ['선택', '브랜드명', '대분류업종', '중분류업종', '창업비용(단위:천원)', '본사업력', '본사부채비율', '본사주요지표', '정보공개', '지도'])
    
    temp_frofit = soup.find_all("span","c prftblGrad")
    temp_fair = soup.find_all("span","c fairGrad")
    temp_growth = soup.find_all("span","c growthGrad")
    temp_safe = soup.find_all("span","c safeGrad")

    temp_frofit_list=list(temp_frofit)
    temp_fair_list=list(temp_fair)
    temp_growth_list=list(temp_growth)
    temp_safe_list=list(temp_safe)

    frofit_count =[str(a).count('color') for a in temp_frofit_list]
    fair_count =[str(a).count('color') for a in temp_fair_list]
    growth_count =[str(a).count('color') for a in temp_growth_list]
    safe_count =[str(a).count('color') for a in temp_safe_list]

    df['수익성'] = frofit_count
    df['공정성'] = fair_count
    df['성장성'] = growth_count
    df['안정성'] = safe_count
    
    
    




In [263]:
# 페이지 넘기면서 표 정보 크롤링하기
for i in range(1,109):
    if i==1:
        crawling_page()
        df_all = df
        time.sleep(3)
    elif i%5 == 1 and i != 1:
        #페이지클릭
        driver.find_element_by_css_selector("#brandCompare > div.body.wType1.forPc > div > div.mPag > a.next").click()
        time.sleep(5)
        #크롤링 
        crawling_page()
        time.sleep(3) 
        df_all = pd.concat([df_all, df])
        time.sleep(3)
    elif i%5 == 2:
        driver.find_element_by_css_selector("#brandCompare > div.body.wType1.forPc > div > div.mPag > a:nth-child(4)").click()
        time.sleep(5)
        crawling_page()
        time.sleep(3) 
        df_all = pd.concat([df_all, df])
        time.sleep(3)
    elif i%5 == 3:
        driver.find_element_by_css_selector("#brandCompare > div.body.wType1.forPc > div > div.mPag > a:nth-child(5)").click()
        time.sleep(5)
        crawling_page()
        time.sleep(3) 
        df_all = pd.concat([df_all, df])
        time.sleep(3)
    elif i%5 == 4:
        driver.find_element_by_css_selector("#brandCompare > div.body.wType1.forPc > div > div.mPag > a:nth-child(6)").click()
        time.sleep(5)
        crawling_page()
        time.sleep(3) 
        df_all = pd.concat([df_all, df])
        time.sleep(3)
    elif i%5 == 0:
        driver.find_element_by_css_selector("#brandCompare > div.body.wType1.forPc > div > div.mPag > a:nth-child(7)").click()
        time.sleep(5)
        crawling_page()
        time.sleep(3) 
        df_all = pd.concat([df_all, df])
        time.sleep(3)
    
    print(df_all.head())
    print(df_all.tail())
    print(len(df_all))
    
len(df_all)

   선택           브랜드명 대분류업종     중분류업종 창업비용(단위:천원) 본사업력 본사부채비율  \
0  선택       이디야커피 추천    외식        커피     129,130  21년    88%   
1  선택         비비큐 추천    외식        치킨      90,789  28년    58%   
2  선택    비비큐(BBQ) 추천    외식        치킨     256,548  28년  1500%   
3  선택  비에이치씨(BHC) 추천    외식        치킨      85,436  19년   205%   
4  선택      배스킨라빈스 추천    외식  아이스크림/빙수     323,350  37년    24%   

                                              본사주요지표 정보공개    지도  수익성  공정성  \
0  수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...    -  지도보기    4    3   
1  수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...    -  지도보기    3    3   
2  수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...    -  지도보기    2    2   
3  수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...    -  지도보기    5    3   
4  수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...    -  지도보기    4    3   

   성장성  안정성  
0    3    3  
1    4    3  
2    3    5  
3    4    4  
4    3    2  
    선택             브랜드명 대분류업종      중분류업종 창업비용(단위:천원) 본사업력 본사부채비율  \


5374

In [264]:
df_all

,선택,브랜드명,대분류업종,중분류업종,창업비용(단위:천원),본사업력,본사부채비율,본사주요지표,정보공개,지도,수익성,공정성,성장성,안정성
0,선택,이디야커피 추천,외식,커피,"129,130",21년,88%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,4,3,3,3
1,선택,비비큐 추천,외식,치킨,"90,789",28년,58%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,3,3,4,3
2,선택,비비큐(BBQ) 추천,외식,치킨,"256,548",28년,1500%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,2,2,3,5
3,선택,비에이치씨(BHC) 추천,외식,치킨,"85,436",19년,205%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,5,3,4,4
4,선택,배스킨라빈스 추천,외식,아이스크림/빙수,"323,350",37년,24%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,4,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,선택,왕짱구 추천,외식,기타 외식,"57,050",18년,3%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,5,3,3,2
20,선택,심민속촌 추천,외식,기타 외식,"395,100",8년,102%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,3,3,2,3
21,선택,오늘게장 추천,외식,한식,"35,100",3년,-3594%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,2,3,0,1
22,선택,모던타코 추천,외식,기타 외식,"19,690",7년,299%,수익성당기순이익 / 총자산 * 100 공정성공정거래위원회의 시정조치 건수 + 민사소...,-,지도보기,3,3,0,4


In [167]:
df_all.to_csv("brand_all.csv")